Import & Parameter

In [ ]:
import random
from collections import defaultdict

# PARAMETER (SAMA DENGAN EXCEL)
NUM_CLASSES = 1200
NUM_MK = 120
NUM_DOSEN = 50
NUM_RUANG = 20
NUM_HARI = 5
NUM_SLOT = 10

POPULATION_SIZE = 50
GENERATIONS = 100
MUTATION_RATE = 0.05
ELITISM = 5

Membuat Jadwal

In [ ]:
def create_chromosome():
    chromosome = []
    for i in range(NUM_CLASSES):
        chromosome.append({
            "kelas": i + 1,
            "mk": f"MK{random.randint(1, NUM_MK)}",
            "dosen": f"D{random.randint(1, NUM_DOSEN)}",
            "ruang": f"R{random.randint(1, NUM_RUANG)}",
            "hari": random.randint(1, NUM_HARI),
            "slot": random.randint(1, NUM_SLOT)
        })
    return chromosome

Menghitung Konflik

In [ ]:
def calculate_conflicts(chromosome):
    konflik_dosen = 0
    konflik_ruang = 0

    dosen_schedule = defaultdict(int)
    ruang_schedule = defaultdict(int)

    for c in chromosome:
        key_dosen = (c["dosen"], c["hari"], c["slot"])
        key_ruang = (c["ruang"], c["hari"], c["slot"])

        dosen_schedule[key_dosen] += 1
        ruang_schedule[key_ruang] += 1

    for count in dosen_schedule.values():
        if count > 1:
            konflik_dosen += count - 1

    for count in ruang_schedule.values():
        if count > 1:
            konflik_ruang += count - 1

    return konflik_dosen, konflik_ruang


Fitness Function

In [ ]:
def fitness(chromosome):
    konflik_dosen, konflik_ruang = calculate_conflicts(chromosome)
    total_konflik = konflik_dosen + konflik_ruang
    return 1 / (1 + total_konflik)

Seleksi – Roulette Wheel

In [2]:
def roulette_wheel_selection(population):
    fitness_values = [fitness(c) for c in population]
    total_fitness = sum(fitness_values)
    pick = random.uniform(0, total_fitness)

    current = 0
    for chromosome, fit in zip(population, fitness_values):
        current += fit
        if current >= pick:
            return chromosome

Crossover – Single Point

In [3]:
def crossover(parent1, parent2):
    point = random.randint(1, NUM_CLASSES - 1)
    child = parent1[:point] + parent2[point:]
    return child

Mutasi

In [4]:
def mutate(chromosome):
    for gene in chromosome:
        if random.random() < MUTATION_RATE:
            gene["hari"] = random.randint(1, NUM_HARI)
            gene["slot"] = random.randint(1, NUM_SLOT)
            gene["ruang"] = f"R{random.randint(1, NUM_RUANG)}"

PROSES EVOLUSI

In [8]:
def genetic_algorithm():
    population = [create_chromosome() for _ in range(POPULATION_SIZE)]

    for gen in range(GENERATIONS):
        population.sort(key=fitness, reverse=True)

        new_population = population[:ELITISM]

        while len(new_population) < POPULATION_SIZE:
            parent1 = roulette_wheel_selection(population)
            parent2 = roulette_wheel_selection(population)
            child = crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)

        population = new_population

        best_fitness = fitness(population[0])
        konflik_dosen, konflik_ruang = calculate_conflicts(population[0])

        print(
            f"Gen {gen+1} | Fitness: {best_fitness:.6f} | "
            f"Konflik Dosen: {konflik_dosen} | Konflik Ruang: {konflik_ruang}"
        )

    return population[0]

MENJALANKAN PROGRAM

In [ ]:
best_schedule = genetic_algorithm()